# PRÁTICA INDEPENDENTE: Otimização de modelos

#### Agora, aplicaremos todos os conceitos que vimos de otimização. Usaremos `ParemeterGrid`, `ParameterSampler`, `GridSearch` e `RandomSearch`. Usaremos também o `dataset` `'diabetes.csv'`.

# Instruções

#### Baixe os pacotes necessários e leia o arquivo `'diabetes.csv'`.

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
import itertools
import random

In [2]:
df= pd.read_csv('diabetes.csv')
df.head()

,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age,label
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


# Exercício 1 - Separação Treino, Teste e Validação

#### Primeiramente, teste a estratégia de dividir em treino, teste e validação. Ou seja, teremos 3 datasets. Siga o passo a passo abaixo.

1. Defina um Pipeline com `kNN` e `MinMaxScaler`. Defina também a validação `StratifiedKFold`;
2. Divida o `df` em `df_train`, `df_test` e `df_val`. Sugestão de percentuais: $70\%$, $15\%$, $15\%$;
3. Crie a rede de parâmetros para o `grid search` (varie métrica de distância, número de vizinhos e pesos de votação);
4. Crie a rede de parâmetros para o `random search` (varie métrica de distância, número de vizinhos e pesos de votação). Use $50$ iterações;
5. Execute um laço para o `grid search`;
6. Execute um loop para o `random search`;
7. Avalie a performance e os conjuntos de hiperparametros associados (lembre-se de que o objetivo é pegar o conjunto, cujo erro de validação é o menor).

#### Dica: Divida primeiro em `df_train` e `df_test` e depois use o `df_train` para dividr em `df_train` e `df_Val`. Lembre de imprimir os parâmetros escolhidos e a pontuação (`score`) obtida. Use sempre `neg_log_loss` como métrica.

In [3]:
#Criando pipeline para KNN
pipeline_knn= Pipeline(steps = [('pre_processor', MinMaxScaler()), 
                             ('knn', KNeighborsClassifier()) 
                            ]
                   )

#Objeto de estratificação dos dados
skf = StratifiedKFold(n_splits = 10, 
                      shuffle = True, 
                      random_state = 42
                     ) 

Split de treino, teste e validação

In [4]:
#Treino e teste
df_train, df_test = train_test_split(df, 
                                   stratify = df['label'], 
                                   test_size = 0.15, 
                                   random_state = 123
                                  )

#Treino e validação
df_train, df_val = train_test_split(df_train, 
                                    stratify = df_train['label'], 
                                    test_size = 0.15, 
                                    random_state = 123
                                   )

In [5]:
X_train, y_train = df_train.drop('label', axis = 1), df_train['label']
X_test, y_test = df_test.drop('label', axis = 1), df_test['label']
X_val, y_val = df_val.drop('label', axis = 1), df_val['label']

print(len(X_train), len(y_train))
print(len(X_test), len(y_test))
print(len(X_val), len(y_val))

554 554
116 116
98 98


GridSearch

In [6]:
#Hiperparametros que serão testados
n_k_grid = list(range(1, 51))

pesos_opc_grid = ['uniform', 'distance']

metricas_distancia_grid= ['euclidean', 'manhattan', 'chebyshev']

In [7]:
#Dicionário com todos os parâmetros para o grid search
param_grid = dict(knn__n_neighbors = n_k_grid, 
                  knn__weights = pesos_opc_grid, 
                  knn__metric = metricas_distancia_grid)

#Todas as combinações possíveis com intertools.product
all_combinations = list(itertools.product(n_k_grid, pesos_opc_grid, metricas_distancia_grid))

print(f'Temos ao todo {len(all_combinations)} combinações possíveis')

Temos ao todo 300 combinações possíveis


In [8]:
#Lista que guardará a acurácia de treino, validação e os parâmetros associados a elas
accuracy_grid = []
y_val_true= y_val


#Acurácia do modelo em treino e validação em todas as combinações de parâmetros possíveis
for combination in list(ParameterGrid(param_grid)):
    
    knn = KNeighborsClassifier(n_neighbors= combination['knn__n_neighbors'], 
                              weights= combination['knn__weights'],
                              metric= combination['knn__metric'])
    
    knn.fit(X_train, y_train)
    
    y_train_pred = knn.predict(X_train)    
    y_val_pred = knn.predict(X_val)
    
    accuracy_train = accuracy_score(y_train_pred, y_train)
    accuracy_val = accuracy_score(y_val_pred, y_val)

    
    accuracy_grid.append([accuracy_train, accuracy_val, combination])

In [9]:
#Dataframe com todas as acurácias de treino e validação para todos os parâmetros testados
df_accuracy_grid= pd.DataFrame(accuracy_grid, columns= ['train_accuracy', 'val_accuracy', 'parameters'])

#Melhores acurácias de treino e validação e seus parâmetros
best_parameters= df_accuracy_grid.sort_values(by= 'val_accuracy', ascending= False).head(1)
train_accuracy= best_parameters['train_accuracy'].values[0]
val_accuracy= best_parameters['val_accuracy'].values[0]
parameters= best_parameters['parameters'].values

print(f'Train Accuracy: {train_accuracy}')
print(f'Validation Accuracy: {val_accuracy}')
print(f'Parameters: {parameters}')

Train Accuracy: 1.0
Validation Accuracy: 0.8061224489795918
Parameters: [{'knn__metric': 'manhattan', 'knn__n_neighbors': 6, 'knn__weights': 'distance'}]


RandomSearch

In [10]:
#Crie a rede de parâmetros para o random search (varie métrica de distância, 
#                                                número de vizinhos e pesos de votação). Use  50  iterações;


n_k_random= list(range(1, 101))

metricas_distancia_random= ['euclidean', 'chebyshev']

pesos_opc_random = ['uniform', 'distance']


#Dicionário com todos os parâmetros para o random search
param_random = dict(knn__n_neighbors = n_k_random, 
                  knn__weights = pesos_opc_random,
                  knn__metric= metricas_distancia_random
                 )

In [11]:
#Lista que guardará as acurácias de treino e validação e os parametros associados a elas no random
accuracy_random = []

y_val_true= y_val

for i in range (1, 100):
    combination= random.choice(list(ParameterGrid(param_random)))
    
    knn = KNeighborsClassifier(n_neighbors= combination['knn__n_neighbors'], 
                              weights= combination['knn__weights'],
                              metric= combination['knn__metric'])
    
    knn = KNeighborsClassifier()
    knn.fit(X_train, y_train)
    y_val_calc = knn.predict(X_train)    
    y_val_pred = knn.predict(X_val)
    
    accuracy_train = accuracy_score(y_val_calc, y_train)
    accuracy_val = accuracy_score(y_val_pred, y_val)

   
    accuracy_random.append([accuracy_train, accuracy_val, combination])

In [12]:
#Dataframe com todas as acurácias de treino e validação para todos os parâmetros testados
df_accuracy_random= pd.DataFrame(accuracy_random, columns= ['train_accuracy', 'val_accuracy', 'parameters'])

#Melhores acurácias de treino e validação e seus parâmetros
best_parameters= df_accuracy_random.sort_values(by= 'val_accuracy', ascending= False).head(1)
train_accuracy= best_parameters['train_accuracy'].values[0]
val_accuracy= best_parameters['val_accuracy'].values[0]
parameters= best_parameters['parameters'].values

print(f'Train Accuracy: {train_accuracy}')
print(f'Validation Accuracy: {val_accuracy}')
print(f'Parameters: {parameters}')

Train Accuracy: 0.7906137184115524
Validation Accuracy: 0.7551020408163265
Parameters: [{'knn__metric': 'chebyshev', 'knn__n_neighbors': 5, 'knn__weights': 'uniform'}]


Combinação de parâmetros com melhor performance foi do Grid com 80% de acurácia no conjunto de validação.

# Exercício 2 - Cross Validation

#### Reaproveitando os passos $1, 2, 3, 4$ do tópico $2$, realize as validações cruzadas com `Grid Search` e `Random Search`. Lembre de imprimir os melhores parâmetros e a prontuação (`score`) obtida.

GridSearchCV

In [13]:
#Instanciando objeto grid realizando o pipeline_knn, com todas as combinações de parâmetros possíveis em param_grid 
# e realizando a estritificação cruzada definida em skf.
grid = GridSearchCV(pipeline_knn,
                    param_grid= param_grid, 
                    cv = skf, 
                    scoring = 'neg_log_loss'
                   )
#Ajustando dados de treino ao grid
grid.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('pre_processor', MinMaxScaler()),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'knn__metric': ['euclidean', 'manhattan', 'chebyshev'],
                         'knn__n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                              12, 13, 14, 15, 16, 17, 18, 19,
                                              20, 21, 22, 23, 24, 25, 26, 27,
                                              28, 29, 30, ...],
                         'knn__weights': ['uniform', 'distance']},
             scoring='neg_log_loss')

In [14]:
#Score e melhores parâmetros do Grid
print('GRID:')
print()
print(f'Score ----> {grid.best_score_}')
print(f'Parâmetros com melhor performance: {grid.best_params_}')

GRID:

Score ----> -0.5102263386078122
Parâmetros com melhor performance: {'knn__metric': 'manhattan', 'knn__n_neighbors': 35, 'knn__weights': 'distance'}


RandomSearchCV

In [15]:
random = RandomizedSearchCV(estimator= pipeline_knn,
                   param_distributions= param_random, 
                    cv = skf, 
                    scoring = 'neg_log_loss',
                    n_iter= 50
                   )
#Ajustando dados de treino ao random
random.fit(X_train, y_train)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
                   estimator=Pipeline(steps=[('pre_processor', MinMaxScaler()),
                                             ('knn', KNeighborsClassifier())]),
                   n_iter=50,
                   param_distributions={'knn__metric': ['euclidean',
                                                        'chebyshev'],
                                        'knn__n_neighbors': [1, 2, 3, 4, 5, 6,
                                                             7, 8, 9, 10, 11,
                                                             12, 13, 14, 15, 16,
                                                             17, 18, 19, 20, 21,
                                                             22, 23, 24, 25, 26,
                                                             27, 28, 29, 30, ...],
                                        'knn__weights': ['uniform',
                                     

In [16]:
#Score e melhores parâmetros do Random
print('RANDOM:')
print()
print(f'Score ----> {random.best_score_}')
print(f'Parâmetros com melhor performance: {random.best_params_}')

RANDOM:

Score ----> -0.5234792868634808
Parâmetros com melhor performance: {'knn__weights': 'distance', 'knn__n_neighbors': 49, 'knn__metric': 'euclidean'}


Como estamos lidando com um métrica que mede o erro, quanto menor melhor. Neste caso estamos usando a métrica neg_log_loss que nos retorna números negativos. Portanto, quanto maior o valor absoluto retornado, porque está em negativo, melhor é o score da combinação de parâmetros em questão.

In [17]:
#Instanciando random search pipeline com a melhor combinação de parâmetros
melhor_knn= random.best_estimator_

In [18]:
#Ajustando modelo aos dados de treino
melhor_knn.fit(X_train, 
             y_train
            )

Pipeline(steps=[('pre_processor', MinMaxScaler()),
                ('knn',
                 KNeighborsClassifier(metric='euclidean', n_neighbors=49,
                                      weights='distance'))])

In [19]:
#Predição no conjunto de teste
y_pred_test= melhor_knn.predict(X_test)

In [20]:
#Acurácia de teste
accuracy_score(y_pred_test, y_test)

0.7931034482758621

#### Avalie a performance no treino e na validação

#### OBS: Não estamos preocupado com performance. A ideia é exercitar a construção desses `steps`. O `dataset` não é tão grande e podemos sofrer com isso.

In [21]:
#Predição no conjunto de treino e validação
y_pred_train= melhor_knn.predict(X_train)
y_pred_val= melhor_knn.predict(X_val)

#Acurácias nos conjuntos de treino e de validação
acuracia_treino= accuracy_score(y_pred_train, y_train)
acuracia_val= accuracy_score(y_pred_val, y_val)

print(f'Acurácia no treino: {acuracia_treino}')
print(f'Acurácia na validação: {acuracia_val}')

Acurácia no treino: 1.0
Acurácia na validação: 0.7448979591836735


Vemos que temos um overffiting, isto é, o modelo entendeu muito bem a dinâmica do conjunto de treino e previu com 100% de acurácia tal conjunto. Porém, como era de se esperar, sua performance diminui quando é apresentado a conjuntos diferentes como o de teste e de validação. 
Contudo, não temos uma acurácia tão ruim assim, sendo a de teste 79% e de validação 74%.